In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from dateutil import parser
from datetime import datetime


# CLEAN dataframe for training
## Scale
* Minutes for arrival
* Time of day (minutes since midnight)
* Day of week (0-6)
* Day of month (0-30)
* Day of the year (0-364)
* Month (0-11)
* Year
* Lat
* Long

## Categorical
* Crime category
* Geo Code



In [2]:
incidents_df = pd.read_csv("data/Police_Incidents_Since_1988.csv", sep = None, dtype={'INCIDENT': np.str_, 'DATE OCCURRED': np.str_, 'TIME OCCURED' : np.str_, 'YEAR OCCURRED': np.int_, 'MONTH OCCURRED': np.int_, 'DATE REPORTED': np.str_, 'TIME ARRIVED': np.str_, 'DATE ARRIVED': np.str_, 'GEO CODE': np.str_, 'HOUSE NUMBER BLOCK': np.str_, 'STREET': np.str_, 'CRIME CODE': np.float64, 'CRIME DESCRIPTION': np.str_, 'CRIME CATEGORY': np.str_, 'CRIME CATEGORY DESCRIPTION': np.str_, 'CSA DESCRIPTION': np.str_, 'PLACE CODE DESCRIPTION': np.str_, 'WEAPONS CODE 1': np.str_, 'WEAPON 1 DESCRIPTION': np.str_, 'WEAPONS CODE 2': np.str_, 'WEAPON 2 DESCRIPTION': np.str_, 'WEAPONS CODE 3': np.str_, 'WEAPON 3 DESCRIPTION': np.str_, 'BIAS CODE': np.float64, 'BIAS DESCRIPTION': np.str_, 'STATUS CODE': np.float64, 'STATUS DESCRIPTION': np.str_, 'COUNTER': int, 'MAPPING ADDRESS': np.str_}, engine = 'python')

In [3]:
#pd.set_option('display.max_columns', None)

#incidents_df.head()

In [4]:
length = len(incidents_df)
latitude = [np.nan] * length
longitude =[np.nan] * length
total_min =[np.nan] * length
day_of_week =[np.nan] * length
day_of_month =[np.nan] * length
day_of_year =[np.nan] * length

time =[np.nan] * length
for index in range(length):
    if index %50000==0:
        print("currently on row ", index, "out of ", length)

    #lat/long
    value = str(incidents_df['MAPPING ADDRESS'][index])
    left_ind = value.find('(')
    mid_ind = value.find(',', left_ind)
    right_ind = value.find(')')
    if left_ind!=-1 and mid_ind !=-1 and right_ind !=-1:
        latitude[index] = float(value[left_ind+1: mid_ind])
        longitude[index] = float(value[mid_ind+1:right_ind])

    arrive = (str(incidents_df['DATE REPORTED'][index]) + ' ' + str(incidents_df['TIME REPORTED'][index]))
    report = (str(incidents_df['DATE ARRIVED'][index]) + ' ' + str(incidents_df['TIME ARRIVED'][index]))
    
    #response time
    try:
        a = parser.parse(arrive)
        b = parser.parse(report)
        delt = b-a
        elapsed = delt.total_seconds()//60
        if elapsed>=0 and elapsed<300:  #ignoring 0 minutes because it was probably not a caller
            time[index] = int(elapsed)
    except:
        pass
    
    #total minutes
    try:
        a = parser.parse(str(incidents_df['DATE REPORTED'][index]))
        
        # day_of_week
        day_of_week[index] = int(a.weekday())
        
        # day_of_month
        day_of_month[index] = int(a.day)
        
        # day_of_year
        day_of_year[index] = int(a.timetuple().tm_yday)
    except:
        pass
    
    try:
        a = parser.parse(incidents_df['TIME REPORTED'][index])
        
        # total_min
        total_min[index] = int(a.hour*60+a.minute)
    except:
        pass
    
print('done!')

incidents_df['LATITUDE'] = latitude
incidents_df['LONGITUDE'] = longitude
incidents_df['RESPONSE TIME'] = time
incidents_df['MINS OF DAY'] = total_min
incidents_df['DAY OF WEEK'] = day_of_week
incidents_df['DAY OF MONTH'] = day_of_month
incidents_df['DAY OF YEAR'] = day_of_year
pd.set_option('display.max_rows', None)
# incidents_df.head()

currently on row  0 out of  474609
currently on row  50000 out of  474609
currently on row  100000 out of  474609
currently on row  150000 out of  474609
currently on row  200000 out of  474609
currently on row  250000 out of  474609
currently on row  300000 out of  474609
currently on row  350000 out of  474609
currently on row  400000 out of  474609
currently on row  450000 out of  474609
done!


## Two dataframes are created:
* cleaned_data.csv has the cleaned data
* cleaned_data_no_zeros.csv has cleaned data where all rows have response time > 0

In [5]:
streamlined_df = incidents_df[['CRIME CATEGORY DESCRIPTION','RESPONSE TIME','MINS OF DAY', 'DAY OF WEEK', 'DAY OF MONTH', 'DAY OF YEAR', 'MONTH OCCURRED', 'YEAR OCCURRED','GEO CODE', 'LATITUDE', 'LONGITUDE']]
streamlined_df = streamlined_df[streamlined_df['RESPONSE TIME'].notna()]
streamlined_df.to_csv('data/cleaned_data.csv')

streamlined_no_zeros_df = streamlined_df[streamlined_df['RESPONSE TIME']>0]
streamlined_no_zeros_df.to_csv('data/cleaned_data_no_zeros.csv')
# streamlined_df.head(10)

204312